In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10460471776228248713, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17320007975257013336
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys

import os
import numpy as np

import keras
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Activation, Dropout,Input, concatenate, Flatten  
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.applications.vgg19 import VGG19
from keras import optimizers, metrics, models, layers
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0
dim_ordering: tf


In [5]:
train_dir = 'data-ISBI-resized-final/train/'
validation_dir = 'data-ISBI-resized-final/validation/'
test_dir = 'data-ISBI-resized-final/test/'
test_augmented_dir = 'data-ISBI-resized-final/test-augmented/'

extracted_features_dir = "extracted_features/"

num_classes = 2
batch_size = 32
img_height, img_width = 380, 380
input_shape = (img_height, img_width, 3)
epochs = 25

In [6]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
#     validation_split= 0.25,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
#     subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
#     subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 59169 images belonging to 2 classes.
Found 18384 images belonging to 2 classes.
Found 966 images belonging to 2 classes.


In [7]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_VGG16_MobileNet_Concatenate3.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_VGG16_MobileNet_Concatenate3.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_VGG16_MobileNet_Concatenate3.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [8]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
         'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
#          'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
#          'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
         'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [12]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

    model = Sequential()
#     model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(num_classes, activation='softmax'))

    #     adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd_opt=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
    rmsprop_opt=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    
    model.compile(optimizer = rmsprop_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2,
                    callbacks=get_callbacks(params))
    
    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    print("_" * 30)
    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]

    print('Test Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))
    print("_" * 50)
    sys.stdout.flush() 
    keras.backend.clear_session()
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [13]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [14]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)
print(best)

Parameters testing:  {'dropout': 0.5, 'lr': 0.0001, 'rho': 1}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 5.1039 - acc: 0.6832 - val_loss: 5.1483 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 5.1063 - acc: 0.6832 - val_loss: 5.1483 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 5.1063 - acc: 0.6832 - val_loss: 5.1483 - val_acc: 0.6806
Epoch 4/25
 - 6s - loss: 5.1063 - acc: 0.6832 - val_loss: 5.1483 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 36us/step
Validation Accuracy: 68.0592%
Validation Loss: 5.148251613811174
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'dropout': 0.6, 'lr': 1e-05, 'rho': 0.5}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6190 - acc: 0.6748 - val_loss: 0.5856 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.5795 - acc: 0.6964 - val_loss: 0.5606 - val_acc: 0.6895
Epoch 3/2

Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.5142 - acc: 0.7648 - val_loss: 0.4912 - val_acc: 0.7768
Epoch 2/25
 - 6s - loss: 0.4931 - acc: 0.7791 - val_loss: 0.4778 - val_acc: 0.7877
Epoch 3/25
 - 6s - loss: 0.4860 - acc: 0.7826 - val_loss: 0.4777 - val_acc: 0.7923
Epoch 4/25
 - 6s - loss: 0.4817 - acc: 0.7842 - val_loss: 0.4757 - val_acc: 0.7923
Epoch 5/25
 - 6s - loss: 0.4770 - acc: 0.7888 - val_loss: 0.4737 - val_acc: 0.7954
Epoch 6/25
 - 6s - loss: 0.4730 - acc: 0.7887 - val_loss: 0.4696 - val_acc: 0.7961
Epoch 7/25
 - 6s - loss: 0.4712 - acc: 0.7916 - val_loss: 0.4609 - val_acc: 0.8003
Epoch 8/25
 - 6s - loss: 0.4681 - acc: 0.7912 - val_loss: 0.4603 - val_acc: 0.8005
Epoch 9/25
 - 6s - loss: 0.4666 - acc: 0.7930 - val_loss: 0.4699 - val_acc: 0.8004
Epoch 10/25
 - 6s - loss: 0.4656 - acc: 0.7942 - val_loss: 0.4860 - val_acc: 0.7841
Epoch 11/25
 - 6s - loss: 0.4637 - acc: 0.7939 - val_loss: 0.4570 - val_acc: 0.8005
Epoch 00011: early stopping
18384/1

Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6175 - acc: 0.6802 - val_loss: 0.5879 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.5825 - acc: 0.6933 - val_loss: 0.5646 - val_acc: 0.6851
Epoch 3/25
 - 6s - loss: 0.5604 - acc: 0.7134 - val_loss: 0.5457 - val_acc: 0.7307
Epoch 4/25
 - 6s - loss: 0.5427 - acc: 0.7382 - val_loss: 0.5315 - val_acc: 0.7569
Epoch 5/25
 - 6s - loss: 0.5303 - acc: 0.7559 - val_loss: 0.5211 - val_acc: 0.7670
Epoch 6/25
 - 6s - loss: 0.5217 - acc: 0.7628 - val_loss: 0.5138 - val_acc: 0.7718
Epoch 7/25
 - 6s - loss: 0.5150 - acc: 0.7700 - val_loss: 0.5087 - val_acc: 0.7742
Epoch 8/25
 - 6s - loss: 0.5106 - acc: 0.7722 - val_loss: 0.5041 - val_acc: 0.7774
Epoch 9/25
 - 6s - loss: 0.5063 - acc: 0.7749 - val_loss: 0.5010 - val_acc: 0.7791
Epoch 10/25
 - 6s - loss: 0.5032 - acc: 0.7767 - val_loss: 0.4987 - val_acc: 0.7789
Epoch 11/25
 - 6s - loss: 0.5011 - acc: 0.7763 - val_loss: 0.4968 - val_acc: 0.7796
Epoch 12/25
 - 6s - loss: 0.4992 - 

Epoch 3/25
 - 7s - loss: 0.4838 - acc: 0.7841 - val_loss: 0.4829 - val_acc: 0.7853
Epoch 4/25
 - 6s - loss: 0.4787 - acc: 0.7870 - val_loss: 0.4798 - val_acc: 0.7877
Epoch 5/25
 - 7s - loss: 0.4740 - acc: 0.7897 - val_loss: 0.4932 - val_acc: 0.7774
Epoch 6/25
 - 7s - loss: 0.4704 - acc: 0.7910 - val_loss: 0.4800 - val_acc: 0.7895
Epoch 7/25
 - 7s - loss: 0.4669 - acc: 0.7926 - val_loss: 0.4688 - val_acc: 0.7943
Epoch 8/25
 - 6s - loss: 0.4639 - acc: 0.7936 - val_loss: 0.4676 - val_acc: 0.7948
Epoch 9/25
 - 7s - loss: 0.4604 - acc: 0.7947 - val_loss: 0.4759 - val_acc: 0.7916
Epoch 10/25
 - 6s - loss: 0.4582 - acc: 0.7959 - val_loss: 0.4637 - val_acc: 0.7985
Epoch 11/25
 - 6s - loss: 0.4564 - acc: 0.7963 - val_loss: 0.4732 - val_acc: 0.7919
Epoch 12/25
 - 6s - loss: 0.4538 - acc: 0.7991 - val_loss: 0.4640 - val_acc: 0.7986
Epoch 13/25
 - 6s - loss: 0.4524 - acc: 0.8001 - val_loss: 0.4617 - val_acc: 0.7980
Epoch 14/25
 - 6s - loss: 0.4510 - acc: 0.8005 - val_loss: 0.4761 - val_acc: 0.7945

Epoch 2/25
 - 6s - loss: 0.4958 - acc: 0.7779 - val_loss: 0.4900 - val_acc: 0.7814
Epoch 3/25
 - 6s - loss: 0.4892 - acc: 0.7819 - val_loss: 0.4862 - val_acc: 0.7836
Epoch 4/25
 - 6s - loss: 0.4845 - acc: 0.7843 - val_loss: 0.4875 - val_acc: 0.7836
Epoch 5/25
 - 6s - loss: 0.4819 - acc: 0.7847 - val_loss: 0.4827 - val_acc: 0.7859
Epoch 6/25
 - 6s - loss: 0.4797 - acc: 0.7871 - val_loss: 0.4790 - val_acc: 0.7874
Epoch 7/25
 - 6s - loss: 0.4777 - acc: 0.7889 - val_loss: 0.4794 - val_acc: 0.7891
Epoch 8/25
 - 6s - loss: 0.4749 - acc: 0.7908 - val_loss: 0.4770 - val_acc: 0.7901
Epoch 9/25
 - 6s - loss: 0.4728 - acc: 0.7902 - val_loss: 0.4744 - val_acc: 0.7910
Epoch 10/25
 - 6s - loss: 0.4704 - acc: 0.7921 - val_loss: 0.4747 - val_acc: 0.7908
Epoch 11/25
 - 6s - loss: 0.4681 - acc: 0.7930 - val_loss: 0.4709 - val_acc: 0.7945
Epoch 12/25
 - 6s - loss: 0.4664 - acc: 0.7941 - val_loss: 0.4696 - val_acc: 0.7960
Epoch 13/25
 - 6s - loss: 0.4648 - acc: 0.7949 - val_loss: 0.4742 - val_acc: 0.7940


...

Epoch 14/25
 - 6s - loss: 0.4624 - acc: 0.7953 - val_loss: 0.4674 - val_acc: 0.7980
Epoch 15/25
 - 6s - loss: 0.4618 - acc: 0.7963 - val_loss: 0.4702 - val_acc: 0.7972
Epoch 16/25
 - 6s - loss: 0.4603 - acc: 0.7978 - val_loss: 0.4691 - val_acc: 0.7985
Epoch 17/25
 - 6s - loss: 0.4597 - acc: 0.7976 - val_loss: 0.4674 - val_acc: 0.7997
Epoch 18/25
 - 6s - loss: 0.4587 - acc: 0.7981 - val_loss: 0.4658 - val_acc: 0.8006
Epoch 19/25
 - 6s - loss: 0.4577 - acc: 0.8004 - val_loss: 0.4694 - val_acc: 0.7997
Epoch 20/25
 - 6s - loss: 0.4570 - acc: 0.8000 - val_loss: 0.4665 - val_acc: 0.7986
Epoch 21/25
 - 6s - loss: 0.4568 - acc: 0.7995 - val_loss: 0.4736 - val_acc: 0.7929
Epoch 00021: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 79.2918%
Validation Loss: 0.4735824263049168
______________________________
Test Accuracy 0.8923395445134575
__________________________________________________
Parameters testing:  {'dropout': 0.5, 'lr': 0.0001, 'rho': 

Parameters testing:  {'dropout': 0.5, 'lr': 1e-07, 'rho': 0.6}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6858 - acc: 0.5524 - val_loss: 0.6626 - val_acc: 0.6806
Epoch 2/25
 - 7s - loss: 0.6684 - acc: 0.5950 - val_loss: 0.6481 - val_acc: 0.6806
Epoch 3/25
 - 6s - loss: 0.6560 - acc: 0.6268 - val_loss: 0.6376 - val_acc: 0.6806
Epoch 4/25
 - 6s - loss: 0.6466 - acc: 0.6475 - val_loss: 0.6301 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 38us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6300985951128993
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'dropout': 0.4, 'lr': 0.001, 'rho': 0.9}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.5103 - acc: 0.7659 - val_loss: 0.5208 - val_acc: 0.7706
Epoch 2/25
 - 6s - loss: 0.4891 - acc: 0.7807 - val_loss: 0.5244 - val_acc: 0.7705
Epoch 3

Epoch 11/25
 - 6s - loss: 0.4594 - acc: 0.7958 - val_loss: 0.4632 - val_acc: 0.7977
Epoch 12/25
 - 6s - loss: 0.4575 - acc: 0.7969 - val_loss: 0.4610 - val_acc: 0.7999
Epoch 13/25
 - 6s - loss: 0.4549 - acc: 0.7981 - val_loss: 0.4617 - val_acc: 0.7977
Epoch 14/25
 - 6s - loss: 0.4525 - acc: 0.7992 - val_loss: 0.4605 - val_acc: 0.8009
Epoch 15/25
 - 6s - loss: 0.4512 - acc: 0.8014 - val_loss: 0.4577 - val_acc: 0.8018
Epoch 16/25
 - 6s - loss: 0.4500 - acc: 0.8010 - val_loss: 0.4644 - val_acc: 0.7987
Epoch 17/25
 - 6s - loss: 0.4480 - acc: 0.8012 - val_loss: 0.4568 - val_acc: 0.8000
Epoch 18/25
 - 6s - loss: 0.4476 - acc: 0.8024 - val_loss: 0.4584 - val_acc: 0.8019
Epoch 19/25
 - 6s - loss: 0.4466 - acc: 0.8025 - val_loss: 0.4577 - val_acc: 0.8028
Epoch 20/25
 - 6s - loss: 0.4454 - acc: 0.8018 - val_loss: 0.4598 - val_acc: 0.8016
Epoch 21/25
 - 6s - loss: 0.4443 - acc: 0.8041 - val_loss: 0.4559 - val_acc: 0.8030
Epoch 22/25
 - 6s - loss: 0.4430 - acc: 0.8039 - val_loss: 0.4532 - val_acc:

Epoch 9/25
 - 6s - loss: 0.4694 - acc: 0.7916 - val_loss: 0.5405 - val_acc: 0.7538
Epoch 10/25
 - 6s - loss: 0.4690 - acc: 0.7914 - val_loss: 0.4719 - val_acc: 0.7985
Epoch 11/25
 - 6s - loss: 0.4672 - acc: 0.7920 - val_loss: 0.4565 - val_acc: 0.8040
Epoch 12/25
 - 6s - loss: 0.4662 - acc: 0.7932 - val_loss: 0.4631 - val_acc: 0.7981
Epoch 13/25
 - 6s - loss: 0.4646 - acc: 0.7959 - val_loss: 0.4934 - val_acc: 0.7811
Epoch 14/25
 - 6s - loss: 0.4628 - acc: 0.7958 - val_loss: 0.4981 - val_acc: 0.7742
Epoch 00014: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 77.4206%
Validation Loss: 0.49811925973136906
______________________________
Test Accuracy 0.8944099378881988
__________________________________________________
Parameters testing:  {'dropout': 0.5, 'lr': 1e-06, 'rho': 0.8}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6372 - acc: 0.6702 - val_loss: 0.6184 - val_acc: 0.6806
Epoch 2/25
 - 6s - loss: 0.6264

Epoch 16/25
 - 6s - loss: 0.4501 - acc: 0.8022 - val_loss: 0.4593 - val_acc: 0.8019
Epoch 17/25
 - 6s - loss: 0.4494 - acc: 0.8011 - val_loss: 0.4684 - val_acc: 0.7976
Epoch 18/25
 - 6s - loss: 0.4473 - acc: 0.8024 - val_loss: 0.4635 - val_acc: 0.8016
Epoch 00018: early stopping
18384/18384 [==============================] - 1s 38us/step
Validation Accuracy: 80.1567%
Validation Loss: 0.46350575484225187
______________________________
Test Accuracy 0.8695652173913043
__________________________________________________
Parameters testing:  {'dropout': 0.7, 'lr': 0.001, 'rho': 0.7}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.5280 - acc: 0.7582 - val_loss: 0.4859 - val_acc: 0.7830
Epoch 2/25
 - 6s - loss: 0.5090 - acc: 0.7743 - val_loss: 0.4812 - val_acc: 0.7866
Epoch 3/25
 - 6s - loss: 0.5048 - acc: 0.7748 - val_loss: 0.4937 - val_acc: 0.7828
Epoch 4/25
 - 6s - loss: 0.5041 - acc: 0.7779 - val_loss: 0.4989 - val_acc: 0.7866
Epoch 5/25
 - 6s - loss: 0.5011 -